## Setup libraries

In [38]:
from math import sqrt
import numpy as np
import pandas as pd
from pandasql import sqldf
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import EarlyStopping

# Setup tables

## Weather

In [39]:
meteo1 = read_csv("WeatherMoggio14-15.csv")
meteo2 = read_csv("WeatherMoggio16-17.csv")
meteo3 = read_csv("WeatherMoggio18-19.csv")
meteo4 = read_csv("WeatherMoggio20-21.csv")

meteo = sqldf('''SELECT *
                 FROM meteo1
                 UNION
                 SELECT *
                 FROM meteo2
                 UNION
                 SELECT *
                 FROM meteo3
                 UNION
                 SELECT *
                 FROM meteo4
                ''')    

print(meteo.shape)
meteo.head()

(2922, 9)


,IDStation,NameStation,Date,Wind_speed,Wind_direction,Temperature,Relative_humidity,Global_radiation,Rainfall
0,111,Cassina Valsassina Moggio,2014-01-01,0.443554,99.0,-0.334965,45.545455,58.097902,0.0
1,111,Cassina Valsassina Moggio,2014-01-02,0.355915,105.0,-0.486111,83.944444,0.000000,7.8
2,111,Cassina Valsassina Moggio,2014-01-03,0.207631,120.0,2.220588,87.042254,30.514085,0.0
3,111,Cassina Valsassina Moggio,2014-01-04,1.024926,175.0,2.137063,89.349650,0.202797,35.8
4,111,Cassina Valsassina Moggio,2014-01-05,0.666721,94.0,2.416084,86.055944,10.951049,25.0


## Pollutants

In [40]:
MoggioAir = read_csv("AirQualityMoggio14-21.csv")
MoggioAir.head()

,IDStation,NameStation,Date,Ammonia,Arsenic,Benzo_a_pyrene,Cadmium,Lead,Nikel,NO,NO2,NOx,Ozone,PM10,PM25
0,681,Moggio Loc Penscei,2014-01-01,0.0,NaN,0.1,NaN,NaN,NaN,0.266667,2.783333,3.583333,76.308333,6.0,7.0
1,681,Moggio Loc Penscei,2014-01-02,0.0,NaN,NaN,NaN,NaN,NaN,0.941667,2.041667,4.883333,64.275000,6.0,5.0
2,681,Moggio Loc Penscei,2014-01-03,0.0,1.1,NaN,0.0,1.1,4.2,0.375000,1.341667,2.183333,71.045833,11.0,4.0
3,681,Moggio Loc Penscei,2014-01-04,0.0,NaN,0.1,NaN,NaN,NaN,0.241667,2.166667,3.491667,65.783333,6.0,7.0
4,681,Moggio Loc Penscei,2014-01-05,0.0,NaN,NaN,NaN,NaN,NaN,0.283333,1.616667,2.158333,72.154167,3.0,8.0


## Joining tables

In [41]:
Moggio = sqldf(''' 
                SELECT MA.IDStation, MA.NameStation, MA.Date, MA.Ammonia, MA.Arsenic, MA.Benzo_a_pyrene, MA.Cadmium, MA.Lead, MA.Nikel,
                MA.NO, MA.NO2, MA.NOx, MA.Ozone, MA.PM10, MA.PM25, ME.Wind_speed, ME.Wind_direction, ME.Temperature, ME.Relative_humidity,
                ME.Global_radiation, ME.Rainfall
                FROM MoggioAir MA JOIN meteo ME
                ON MA.Date = ME.Date
                ''')

Moggio.head()

,IDStation,NameStation,Date,Ammonia,Arsenic,Benzo_a_pyrene,Cadmium,Lead,Nikel,NO,...,NOx,Ozone,PM10,PM25,Wind_speed,Wind_direction,Temperature,Relative_humidity,Global_radiation,Rainfall
0,681,Moggio Loc Penscei,2014-01-01,0.0,NaN,0.1,NaN,NaN,NaN,0.266667,...,3.583333,76.308333,6.0,7.0,0.443554,99.0,-0.334965,45.545455,58.097902,0.0
1,681,Moggio Loc Penscei,2014-01-02,0.0,NaN,NaN,NaN,NaN,NaN,0.941667,...,4.883333,64.275000,6.0,5.0,0.355915,105.0,-0.486111,83.944444,0.000000,7.8
2,681,Moggio Loc Penscei,2014-01-03,0.0,1.1,NaN,0.0,1.1,4.2,0.375000,...,2.183333,71.045833,11.0,4.0,0.207631,120.0,2.220588,87.042254,30.514085,0.0
3,681,Moggio Loc Penscei,2014-01-04,0.0,NaN,0.1,NaN,NaN,NaN,0.241667,...,3.491667,65.783333,6.0,7.0,1.024926,175.0,2.137063,89.349650,0.202797,35.8
4,681,Moggio Loc Penscei,2014-01-05,0.0,NaN,NaN,NaN,NaN,NaN,0.283333,...,2.158333,72.154167,3.0,8.0,0.666721,94.0,2.416084,86.055944,10.951049,25.0


# Quadrants elaboration

In [42]:
#Primo quadrante
Q1 = Moggio.loc[((Moggio.Wind_direction > 0) &
                 (Moggio.Wind_direction < 90)), :]

a = Q1.shape[0]
arr1 = []
arr1 = [1 for i in range(a)]

del Q1['Wind_direction']
Q1["Wind_direction"] = arr1
columns_titles = ["index", "Date","IDStation","NameStation", "Ammonia", "Arsenic", "Benzo_a_pyrene", "Cadmium", "Lead", "Nikel", "NO", "NO2", "NO2",
                  "NOx", "Ozone", "PM10", "PM25", "Wind_speed", "Wind_direction", "Temperature", "Relative_humidity", "Global_radiation", "Rainfall" ]
                  
Q1 = Q1.reindex(columns=columns_titles)
Q1 = Q1.drop("index", axis = 1)

#Secondo quadrante
Q2 = Moggio.loc[((Moggio.Wind_direction > 90) &
                 (Moggio.Wind_direction < 180)), :]
a = Q2.shape[0]
arr2 = []
arr2 = [2 for i in range(a)]

del Q2['Wind_direction']
Q2["Wind_direction"] = arr2

#Terzo quadrante
Q3 = Moggio.loc[((Moggio.Wind_direction > 180) &
                 (Moggio.Wind_direction < 270)), :]

a = Q3.shape[0]
arr3 = []
arr3 = [3 for i in range(a)]

del Q3['Wind_direction']
Q3["Wind_direction"] = arr3

#Quarto quadrante
Q4 = Moggio.loc[((Moggio.Wind_direction > 270) &
                 (Moggio.Wind_direction < 360)), :]

a = Q4.shape[0]
arr4 = []
arr4 = [4 for i in range(a)]

del Q4['Wind_direction']
Q4["Wind_direction"] = arr4

Moggio = sqldf('''
SELECT *
FROM Q1
UNION
SELECT *
FROM Q2
UNION
SELECT *
FROM Q3
UNION
SELECT *
FROM Q4
''')


Moggio.rename(columns={'Rainfall': 'Quadrante'}, inplace=True)

Moggio

/var/folders/m5/1xdj8pfj2cs3wjpc41g9qkj80000gn/T/ipykernel_9423/4078781200.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Q1["Wind_direction"] = arr1
/var/folders/m5/1xdj8pfj2cs3wjpc41g9qkj80000gn/T/ipykernel_9423/4078781200.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Q2["Wind_direction"] = arr2
/var/folders/m5/1xdj8pfj2cs3wjpc41g9qkj80000gn/T/ipykernel_9423/4078781200.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

,Date,IDStation,NameStation,Ammonia,Arsenic,Benzo_a_pyrene,Cadmium,Lead,Nikel,NO,...,NOx,Ozone,PM10,PM25,Wind_speed,Wind_direction,Temperature,Relative_humidity,Global_radiation,Quadrante
0,681,Moggio Loc Penscei,2014-01-01,0.0,NaN,0.1,NaN,NaN,NaN,0.266667,...,3.583333,76.308333,6.0,7.0,0.443554,-0.334965,45.545455,58.097902,0.000000,2.0
1,681,Moggio Loc Penscei,2014-01-02,0.0,NaN,NaN,NaN,NaN,NaN,0.941667,...,4.883333,64.275000,6.0,5.0,0.355915,-0.486111,83.944444,0.000000,7.800000,2.0
2,681,Moggio Loc Penscei,2014-01-03,0.0,1.1,NaN,0.0,1.1,4.2,0.375000,...,2.183333,71.045833,11.0,4.0,0.207631,2.220588,87.042254,30.514085,0.000000,2.0
3,681,Moggio Loc Penscei,2014-01-04,0.0,NaN,0.1,NaN,NaN,NaN,0.241667,...,3.491667,65.783333,6.0,7.0,1.024926,2.137063,89.349650,0.202797,35.800000,2.0
4,681,Moggio Loc Penscei,2014-01-05,0.0,NaN,NaN,NaN,NaN,NaN,0.283333,...,2.158333,72.154167,3.0,8.0,0.666721,2.416084,86.055944,10.951049,25.000000,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2874,2021-12-21,681,Moggio Loc Penscei,NaN,NaN,NaN,NaN,NaN,NaN,1.283333,...,7.858333,59.812500,14.0,11.0,0.388929,1.000000,-0.059722,76.131944,7.833333,0.0
2875,2021-12-27,681,Moggio Loc Penscei,NaN,NaN,NaN,NaN,NaN,NaN,1.308333,...,5.445833,65.741667,9.0,3.0,0.083350,1.000000,3.250694,86.062500,20.361111,0.0
2876,2021-12-29,681,Moggio Loc Penscei,NaN,NaN,NaN,NaN,NaN,NaN,1.279167,...,3.012500,81.395833,2.0,3.0,0.618074,1.000000,7.252083,52.291667,46.743056,0.0
2877,2021-12-30,681,Moggio Loc Penscei,NaN,NaN,NaN,NaN,NaN,NaN,1.295833,...,3.083333,77.762500,1.0,1.0,0.539214,1.000000,12.413194,42.430556,55.104167,0.0


In [43]:
Moggio

,Date,IDStation,NameStation,Ammonia,Arsenic,Benzo_a_pyrene,Cadmium,Lead,Nikel,NO,...,NOx,Ozone,PM10,PM25,Wind_speed,Wind_direction,Temperature,Relative_humidity,Global_radiation,Quadrante
0,681,Moggio Loc Penscei,2014-01-01,0.0,NaN,0.1,NaN,NaN,NaN,0.266667,...,3.583333,76.308333,6.0,7.0,0.443554,-0.334965,45.545455,58.097902,0.000000,2.0
1,681,Moggio Loc Penscei,2014-01-02,0.0,NaN,NaN,NaN,NaN,NaN,0.941667,...,4.883333,64.275000,6.0,5.0,0.355915,-0.486111,83.944444,0.000000,7.800000,2.0
2,681,Moggio Loc Penscei,2014-01-03,0.0,1.1,NaN,0.0,1.1,4.2,0.375000,...,2.183333,71.045833,11.0,4.0,0.207631,2.220588,87.042254,30.514085,0.000000,2.0
3,681,Moggio Loc Penscei,2014-01-04,0.0,NaN,0.1,NaN,NaN,NaN,0.241667,...,3.491667,65.783333,6.0,7.0,1.024926,2.137063,89.349650,0.202797,35.800000,2.0
4,681,Moggio Loc Penscei,2014-01-05,0.0,NaN,NaN,NaN,NaN,NaN,0.283333,...,2.158333,72.154167,3.0,8.0,0.666721,2.416084,86.055944,10.951049,25.000000,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2874,2021-12-21,681,Moggio Loc Penscei,NaN,NaN,NaN,NaN,NaN,NaN,1.283333,...,7.858333,59.812500,14.0,11.0,0.388929,1.000000,-0.059722,76.131944,7.833333,0.0
2875,2021-12-27,681,Moggio Loc Penscei,NaN,NaN,NaN,NaN,NaN,NaN,1.308333,...,5.445833,65.741667,9.0,3.0,0.083350,1.000000,3.250694,86.062500,20.361111,0.0
2876,2021-12-29,681,Moggio Loc Penscei,NaN,NaN,NaN,NaN,NaN,NaN,1.279167,...,3.012500,81.395833,2.0,3.0,0.618074,1.000000,7.252083,52.291667,46.743056,0.0
2877,2021-12-30,681,Moggio Loc Penscei,NaN,NaN,NaN,NaN,NaN,NaN,1.295833,...,3.083333,77.762500,1.0,1.0,0.539214,1.000000,12.413194,42.430556,55.104167,0.0


# Exporting data

In [44]:
Moggio.to_csv(
    '/Users/marcovinciguerra/Github/GitTesi/Python/Moggio/QuadrantiNuovo2/QuadrantiMoggio2.csv')

